# 03. Mock Photon Comparison

In this Notebook we compare the mock photon propagation with the existing methods like the normal flow. To do that, we generate a line detector and a couple of events that we propagate using each propagator.

In [1]:
import pandas as pd

from ananke.configurations.detector import DetectorConfiguration
from ananke.services.detector import DetectorBuilderService
from olympus.event_generation.generators import GeneratorCollection, GeneratorFactory
from olympus.event_generation.medium import MediumEstimationVariant, Medium
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.event_generation.photon_propagation.norm_flow_photons import NormalFlowPhotonPropagator

/home/janik/projects/master/env/lib/python3.8/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "
2023-01-10 13:45:22.809994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


Let's define the detector next:

In [2]:
oms_per_line = 3
dist_z = 50  # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100  # m
pmts_per_module = 16
pmt_cath_area_r = 75e-3 / 2  # m
module_radius = 0.21  # m
efficiency = 0.42 # Christian S. Number

detector_configuration = DetectorConfiguration.parse_obj(
    {
        "string": {
            "module_number": oms_per_line,
            "module_distance": dist_z
        },
        "pmt": {
            "efficiency": efficiency,
            "noise_rate": dark_noise_rate,
            "area": pmt_cath_area_r
        },
        "module": {
            "radius": module_radius
        },
        "geometry": {
            "type": "single",
        },
        "seed": 31338
    }
)

detector_service = DetectorBuilderService()
det = detector_service.get(configuration=detector_configuration)

Next up we generate our events:

In [3]:
medium = Medium(MediumEstimationVariant.PONE_OPTIMISTIC)

generator_factory = GeneratorFactory(det)

cascades_generator = generator_factory.create(
    "cascade", particle_id=11, log_minimal_energy=2, log_maximal_energy=5.5, rate=0.05
)

records = cascades_generator.generate_records(
    number_of_samples=2
)

sources = cascades_generator.propagate(records)

records.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id
0,26.293626,-1.990358,-53.699838,0.575173,-0.311980,0.756204,-6440121634775625235,118401.349187,3000.0,0.0,cascade,11
1,28.460169,18.393233,5.517846,0.683393,-0.370679,-0.628944,-6440120818731838995,446.013358,3000.0,0.0,cascade,11


Now we need our photon propagators

In [4]:
mock_photon_propagator = MockPhotonPropagator(
    detector=det,
    medium=medium,
    angle_resolution=180,
)

normal_photon_propagator = NormalFlowPhotonPropagator(
    detector=det,
    shape_model_path="../../hyperion/data/photon_arrival_time_nflow_params.pickle",
    counts_model_path="../../hyperion/data/photon_arrival_time_counts_params.pickle",
    medium=medium
)

/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


Now lets Propagate:

In [5]:
mock_hits = mock_photon_propagator.propagate(records, sources)

mock_hits.df.head()

,time,string_id,module_id,pmt_id,record_id
0,2.059627,0,0,0,-6440121634775625235
1,1.816164,0,0,0,-6440121634775625235
2,8.108992,0,0,0,-6440121634775625235
0,7.001629,0,0,0,-6440121634775625235
1,2.921966,0,0,0,-6440121634775625235


And doing the same with the normal photons.

In [6]:
normal_hits = normal_photon_propagator.propagate(records, sources)

normal_hits.df.head()

/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


,time,string_id,module_id,pmt_id,record_id
0,14.203907,0,0,0,-6440121634775625235
1,86.173187,0,0,0,-6440121634775625235
2,149.561005,0,0,0,-6440121634775625235
3,6.382759,0,0,0,-6440121634775625235
4,8.209618,0,0,0,-6440121634775625235


## Comparison of the photon propagators

Now that we have all the hits we want we can compare the following cases:

1. number of hits per module
2. hit arrival times


### Number of hits per module

Let's have a look at the mock propagation:

In [7]:
aggregated_normal_hits = mock_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

time   pmt_id  record_id
string_id module_id                             
0         0          2439465  2439465    2439465
          1          2834299  2834299    2834299
          2          2018930  2018930    2018930

In [8]:
aggregated_normal_hits = normal_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id
string_id module_id                          
0         0          64464   64464      64464
          1          16465   16465      16465
          2           4117    4117       4117